In [6]:
import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Optional
from transformers import pipeline
import cv2

In [8]:
@dataclass
class StoryPanel:
    text: str
    image: Optional[np.ndarray] = None
    position: int = 0

In [10]:
class Story:
    def __init__(self):
        self.text_analyzer = pipeline("text-classification")
        self.panels: List[StoryPanel] = []

In [12]:
    def add_panel(self, text: str) -> StoryPanel:
        panel = StoryPanel(text = text, position = len(self.panels))
        self.panels.append(panel)
        return panel

In [14]:
    def analyze_scene(self, text: str) -> Dict:
        elements = {
            'mood': self.extract_mood(text),
            'subjects': self._identify_subjects(text),
            'actions': self._parse_actions(text),
            'scene_type': self._determine_scene_type(text)
        }
        return elements

In [24]:
    def _extract_mood(self, text: str) -> str:
        mood_keywords = {
            'dramatic': ['intense', 'dramatic', 'suspense'],
            'action': ['fight', 'chase', 'battle'],
            'calm': ['peaceful', 'quiet', 'gentle']
        }
        text = text.lower()
        for mood, keywords in mood_keywords.items():
            if any(keyword in text for keyword in keywords):
                return mood
        return 'neutral'

In [26]:
    def _identify_subjects(self, text: str) -> List[str]:
        return [word for word in text.split()
                if word.istitle() and word.isalpha()]

In [28]:
    def _parse_actions(self, text: str) -> List[str]:
        action_verbs = ['runs', 'jumps', 'fights', 'stands', 'looks']
        return [word for word in text.split()
            if word.lower() in action_verbs]

In [30]:
    def _determine_scene_type(self, text: str) -> str:
        scene_types = {
            'closeup': ['face', 'expression', 'eyes'],
            'wide': ['landscape', 'building', 'sky'],
            'action': ['battle', 'chase', 'fight']
        }
        text = text.lower()
        for scene_type, keywords in scene_types.items():
            if any(keyword in text for keyword in keywords):
                return scene_type
        return 'medium'

In [32]:
    def generate_layout(self) -> List[Dict]:
        layouts = []
        for panel in self.panels:
            scene_elements = self.analyze_scene(panel.text)
            layout = {
                'panel_id': panel.position,
                'description': panel.text,
                'scene_type': scene_elements['scene_type'],
                'mood': scene_elements['mood'],
                'composition': self._suggest_composition(scene_elements)
            }
            layouts.append(layout)
        return layouts

In [34]:
    def _suggest_composition(self, scene_elements: Dict) -> Dict:
        composition = {
            'aspect_ratio': '16:9', #Favorite ratio
            'camera_angle': 'eye_level',
            'framing': 'medium'
        }
        if scene_elements['scene_type'] == 'closeup':
            composition['framing'] = 'close'
        elif scene_elements['scene_type'] == 'wide':
            composition['framing'] = 'wide'
            composition['camera_angle'] = 'high'

        if scene_elements['mood'] == 'dramatic':
            composition['camera_angle'] = 'low'
        return composition

In [36]:
def main():
    generator = Story()
    generator.add_panel("A hero stands atop a tall building, cape flowing in the wind")
    generator.add_panel("Camera zooms in on their determined expression")
    generator.add_panel("They spot trouble in the distance")
    layouts = generator.generate_layout()
    for i in layouts:
        print(f"\nPanel {i['panel_id'] + 1}:")
        print(f"Description: {i['description']}:")
        print(f"Scene Type: {i['scene_type']}:")
        print(f"Mood: {i['mood']}")
        print("Composition:", i['panel_id'])

In [38]:
if __name__ == "__main__":
    main()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.